In [3]:
import pandas as pd
import numpy as np
import sys
import math 
import seaborn as sea
import datetime as dt
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

In [4]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [5]:
selected = dataset[['cod_location','date_id','total_cases']]
selected.head()

,cod_location,date_id,total_cases
0,9,20200313,2.0
1,9,20200319,0.0
2,9,20200320,4.0
3,9,20200321,0.0
4,9,20200322,0.0


In [6]:
selected = selected.groupby('cod_location').tail(5)

In [7]:
selected.head()

,cod_location,date_id,total_cases
183,9,20200917,3328.0
184,9,20200918,3382.0
185,9,20200919,3460.0
186,9,20200920,3460.0
187,9,20200921,3551.0


In [8]:
#Input na rede neural
selected.to_csv('input.csv')

In [9]:
def aver(input):
    medias = input.groupby(['cod_location'])['total_cases'].mean()
    return medias

In [10]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [11]:
def range(init, end, step=1):
    start = float(init)
    stop = float(end)
    step = float(step)
    
    result = [start]
    curr = start
    while curr < stop:
        curr += step
        result.append(curr)
    return result

In [12]:
#Funções de mebramento
def up(a, b, x):
    a = float(a)
    b = float(b)
    x = float(x)
    if x < a:
        return 0
    if x < b:
        return (x - a) / (b - a)
    return 1.0

In [13]:
def down(a, b, x):
    return 1. - up(a, b, x)

In [14]:
def tri(a, b, x):
    a = float(a)
    b = float(b)
    m = (a + b) / 2.
    first = (x - a) / (m - a)
    second = (b - x) / (b - m)
    return max(min(first, second), 0.)

In [15]:
def trap(a, b, c, d, x):
    first = (x - a) / (b - a)
    second = (d - x) / (d - c)
    return max(min(first, 1., second), 0.)

In [16]:
def edge(p, valor):
    if not p:
        return 0.0
    return math.pow(valor,p)
muito = partial(edge,2.)
extremamente = partial(edge,3.)
pouco = partial(edge,0.5)
levemente = partial(edge, 1. /3.)

In [17]:
def fuzzyficacao(dominio,tipo):
    domin_sz = float(len(dominio))
    delta = lambda x: x if (x < 0.5) else (1.0 - x)
    result = (2. / domin_sz) 
    return result

In [18]:
def aprox(fuzzy, n, dominio):
    hw = fuzzy * (max(dominio) - min(dominio))
    return  partial(tri, n - hw, n + hw)

In [20]:
########################USAR PARA TESTES, DEPOIS SUBSTITUIR PELOS DADOS DO DATASET#############################################
companies = [
    ('a', 500, 7), ('b', 600, -9), ('c', 800, 17),
    ('d', 850, 12), ('e', 900, -11), ('f', 1000, 15),
    ('g', 1100, 14), ('h', 1200, 1), ('i', 1300, -2),
    ('j', 1400, -6), ('k', 1500, 12)
]
profit = itemgetter(2)
sales = itemgetter(1)

percentages = map(float, range(-10, 30, 1))
profitable = partial(up, 0., 15.)
high = partial(up, 600., 1150.)

fand = min

def filtro(predicado, itens):
    snd = itemgetter(1)
    return filtro(
        lambda x: snd(x) != 0.0,
        map(predicado, itens)
    )

def p1(company):
    value = profitable(profitable(profit(company)))
    return (company, fand(value,1))

def p2(company):
    a = profitable(profit(company))
    b = high(sales(company))
    return (company,fand(a,b))

def p3(company):
    a = somewhat(profitable(profit(company)))
    b = very(high(sales(company)))
    return (company,fand(a,b))

# exemplo 
sizes = range(4,13,0.5)

short = partial(down, 1.5, 1.625)
medium = partial(tri, 1.525, 1.775)
tall = partial(tri, 1.675, 1.925)
very_tall = partial(up, 1.825, 1.95)

def small(size):
    return down(4., 6., size)


#average = partial(tri, 5., 9.)
def average(size):
    return tri(5., 9., size)


#big = partial(tri, 8., 12.)
def big(size):
    return tri(8., 12., size)


#very_big = partial(up, 11., 13.)
def very_big(size):
    return up(11., 13., size)

#fl.near(20, fl.range(0, 40, 1))(17.5)
near = partial(aprox, 0.125)
around = partial(aprox, 0.25)
roughly = partial(aprox, 0.375)


rules = [
    (short, small),
    (medium, average),
    (tall, big),
    (very_tall, very_big)
]
################################################################################################################################

In [21]:
def func_att(val,func, sz):
    prim = func(sz)
    return (val * prim)